<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo">


Model Parallelism with SKLearn and Dask
=======================


<img src="https://avatars2.githubusercontent.com/u/365630?v=3&s=400"
     align="right"
     width="25%"
     alt="SKLearn logo">


*How do we choose the right parameters for a machine learning pipeline?*

This notebook takes a standard example from the Scikit-Learn documententation and parallelizes it using a Dask-powered `GridSearchCV` function, which is a drop in replacement.  We achieve significant speedup on a cluster on an important problem just by changing an import.


### SKLearn example

Taken from: http://scikit-learn.org/stable/auto_examples/plot_digits_pipe.html#sphx-glr-auto-examples-plot-digits-pipe-py

In [ ]:
# Code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause


import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# from dask_searchcv import GridSearchCV  # <<---- We add this line

logistic = linear_model.LogisticRegression()

pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

digits = datasets.load_digits()
X_digits = digits.data
y_digits = digits.target

In [ ]:
%%time

n_components = [20, 40, 64]
Cs = np.logspace(-4, 4, 3)

#Parameters of pipelines can be set using ‘__’ separated parameter names:

estimator = GridSearchCV(pipe,
                         dict(pca__n_components=n_components,
                              logistic__C=Cs))
estimator.fit(X_digits, y_digits)

In [ ]:
estimator.best_params_

### Connect to distributed cluster

Up until now everything was done on this local machine.

Now we connect to a cluster and run the above cells again.  Given our new scale we can increase the parameter grid considerably and still run quickly.

In [ ]:
from dask.distributed import Client
c = Client('localhost:8786')
c